### Nuclear Trade Atlas Data 2017-2021

Looking at the Nuclear Nuclear Revealed Comparative Advantage (NRCA) by Country.

Data by the European Commission, Joint Research Centre

In [87]:
import pandas as pd
import json
from networkx import random_geometric_graph, set_node_attributes,betweenness_centrality
from pyvis.network import Network
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [88]:
nuclear_com_df = pd.read_csv('data/NUCLEAR_COMMODITIES_RCA_DATA_2023.csv')
nuclear_trade_df = pd.read_csv('data/NUCLEAR_TRADE_ATLAS_DATA_2023.csv')
nuclear_total_df = pd.read_csv('data/TOTAL_TRADE_YEAR_2023.csv')

**NUCLEAR_COMMODITIES_RCA_DATA_2023**

(36210 x 10)

The estimation of the Revealed Comparative Advantage by countries in the trade of nuclear commodities for the time period of reference. 
- REPORTER (country reporting trade, expressed in full text name, as well as ISO2 and ISO3 country codes)
- FLOW of trade (import, export)
- PERIOD of trade (a year)
- NUCLEAR COMMODITY DESCRIPTION
- RCA VALUE USD (the Revealed Comparative Advantage measured on the value of trade)
- RCA QUANTITY KG (the Revealed Comparative Advantage measured on the quantity of trade)
- NRCA VALUE USD (the Normalized Revealed Comparative Advantage measured on the value of trade)
- NRCA QUANTITY KG (the Normalized Revealed Comparative Advantage measured on the quantity of trade)

**NUCLEAR_TRADE_ATLAS_DATA_2023**

(96210 x 13)

Trade flow data between countries in nuclear commodities over time. 
- REPORTER (country reporting trade, expressed in full text name, as well as ISO2 and ISO3 country codes)
- PARTNER (country partner in trade, expressed in full text name, as well as ISO2 and ISO3 country codes)
- FLOW of trade (import, export)
- PERIOD of trade (a year)
- COMMODITY CODE and COMMODITY DESCRIPTION (expressed in the Harmonized System)
- NUCLEAR COMMODITY DESCRIPTION
- Trade VALUE USD and QUANTITY KG.

**TOTAL_TRADE_YEAR_2023**

(2261 x 7)

Trade measures by countries in all commodities (nuclear and non strategic). This is of use to compute the share of nuclear trade versus all trade. Total trade by countries measured on the aggregation of all commodities. 
- REPORTER (country reporting trade, expressed in full text name, as well as ISO2 and ISO3 country codes)
- FLOW of trade (import, export)
- PERIOD of trade (a year)
- Trade VALUE USD and QUANTITY KG.

In [89]:
nuclear_com_df.head(5)

,FLOW,NUCLEAR COMMODITY DESCRIPTION,PERIOD,REPORTER,REPORTER ISO2,REPORTER ISO3,NRCA QUANTITY KG,NRCA VALUE USD,RCA QUANTITY KG,RCA VALUE USD
0,Export,Beryllium,12/31/2017,Andorra,AD,AND,-1.0,-1.0,0.0,0.0
1,Export,Beryllium,12/31/2017,United Arab Emirates,AE,ARE,-1.0,-1.0,0.0,0.0
2,Export,Beryllium,12/31/2017,Afghanistan,AF,AFG,-1.0,-1.0,0.0,0.0
3,Export,Beryllium,12/31/2017,Antigua and Barbuda,AG,ATG,-1.0,-1.0,0.0,0.0
4,Export,Beryllium,12/31/2017,Albania,AL,ALB,-1.0,-1.0,0.0,0.0


In [90]:
nuclear_com_df.REPORTER.value_counts()

Andorra              170
Mexico               170
Mozambique           170
Namibia              170
New Caledonia        170
                    ... 
Greenland            170
Gambia               170
Guinea               170
Equatorial Guinea    170
Zimbabwe             170
Name: REPORTER, Length: 213, dtype: int64

In [91]:
nuclear_trade_df.head(5)

,COMMODITY CODE,COMMODITY DESCRIPTION,FLOW,NUCLEAR COMMODITY DESCRIPTION,PARTNER,PARTNER ISO2,PARTNER ISO3,PERIOD,REPORTER,REPORTER ISO2,REPORTER ISO3,QUANTITY KG,VALUE USD
0,2844.40,"Radioactive elements, isotopes and compounds, ...",Export,Radioactive elements and isotopes,Libya,LY,LBY,12/31/2017,Tunisia,TN,TUN,231,69653
1,8401.30,"Fuel elements ""cartridges"", non-irradiated, in...",Export,"Fuel elements, non-irradiated",Sudan,SD,SDN,12/31/2017,Egypt,EG,EGY,0,30
2,8101.99,"Articles of tungsten, n.e.s.",Export,Tungsten articles,Niger,NE,NER,12/31/2017,Algeria,DZ,DZA,90,164
3,8101.99,"Articles of tungsten, n.e.s.",Export,Tungsten articles,Ethiopia,ET,ETH,12/31/2017,Egypt,EG,EGY,1,32
4,2844.40,"Radioactive elements, isotopes and compounds, ...",Export,Radioactive elements and isotopes,Algeria,DZ,DZA,12/31/2017,South Africa,ZA,ZAF,10,53930


In [92]:
nuclear_trade_df.REPORTER.value_counts()

USA                         6186
Germany                     5239
Netherlands                 4469
China                       4144
United Kingdom              3750
                            ... 
FS Micronesia                  1
Tonga                          1
Wallis and Futuna Isds         1
Falkland Isds (Malvinas)       1
Guinea-Bissau                  1
Name: REPORTER, Length: 213, dtype: int64

In [93]:
nuclear_total_df.head(5)

,FLOW,PERIOD,REPORTER,REPORTER ISO2,REPORTER ISO3,QUANTITY KG,VALUE USD
0,Export,12/31/2017,Algeria,DZ,DZA,94831002298,39517164846
1,Export,12/31/2017,Egypt,EG,EGY,45735370832,33513162526
2,Export,12/31/2017,Libya,LY,LBY,43666980822,18175762311
3,Export,12/31/2017,Morocco,MA,MAR,38055569007,33726889681
4,Export,12/31/2017,Tunisia,TN,TUN,9686988135,16508281515


In [94]:
nuclear_total_df.REPORTER.value_counts()

Algeria                  10
Lithuania                10
Rep. of Korea            10
Singapore                10
Sri Lanka                10
                         ..
Nicaragua                10
Panama                   10
Saint Barthélemy         10
Saint Kitts and Nevis    10
Neth. Antilles            1
Name: REPORTER, Length: 227, dtype: int64

In [95]:
nuclear_trade_df = nuclear_trade_df[nuclear_trade_df.PERIOD=='12/31/2021']
nuclear_trade_df = nuclear_trade_df[['REPORTER', 'PARTNER', 'FLOW', 'VALUE USD']]
nuclear_trade_df

,REPORTER,PARTNER,FLOW,VALUE USD
77188,Tunisia,Libya,Export,90412
77189,Tunisia,Algeria,Export,183
77190,Egypt,Cote d'Ivoire,Export,3873
77191,Egypt,Nigeria,Export,30148
77192,Egypt,South Africa,Export,9722
...,...,...,...,...
96205,Fiji,Australia,Import,2221
96206,Papua New Guinea,Australia,Import,6242
96207,Papua New Guinea,New Zealand,Import,2759
96208,Samoa,New Zealand,Import,127


In [98]:
nuclear_trade_df_b = nuclear_trade_df.sort_values(by='VALUE USD', ascending=False)[:1000]

In [99]:
G = nx.DiGraph()

# Iterate over the rows of the DataFrame and add edges to the graph
for _, row in nuclear_trade_df_b.iterrows():
    reporter = row['REPORTER']
    partner = row['PARTNER']
    flow = row['FLOW']
    value_usd = row['VALUE USD']

    # Add the edge with the 'VALUE USD' as the weight and 'FLOW' as an attribute
    G.add_edge(reporter, partner, weight=value_usd, flow=flow)

# Calculate betweenness centrality
bb = nx.betweenness_centrality(G)

# Set node sizes based on betweenness centrality
node_sizes = {k: 50 * np.sqrt(v) for k, v in bb.items()}

# Calculate total weight for each node based on incoming edges
node_weights = G.in_degree(weight='weight')

# Set node attributes
set_node_attributes(G, node_sizes, 'node_size')
set_node_attributes(G, node_weights, 'weight')

# Create a network visualization
# fc = Network(notebook=True, width='800px', height='600px')
fc = Network(notebook=True, height="750px", width="100%", bgcolor="#222222", font_color="white")
fc.repulsion()

# Add nodes and edges to the network visualization
for node in G.nodes():
    node_label = f"{node}\nValue: {G.nodes[node]['weight']}"
    fc.add_node(node, size=G.nodes[node]['node_size'], title=node_label)

for edge in G.edges():
    fc.add_edge(edge[0], edge[1], value=G.edges[edge]['weight'], title=G.edges[edge]['flow'])

# Set the title for the graph
# fc.set_options("""
# var options = {
#   "title": {
#     "text": "Nuclear Trade Network"
#   }
# }
# network.setOptions(options);
# """)

# Display the network visualization
fc.show('graph.html')

graph.html


In [43]:
# convert to json
nuclear_trade_json = nuclear_trade_df
nuclear_trade_json = nuclear_trade_json.to_dict(orient='records')

with open('data/nuclear_atlas.json', 'w') as json_file:
    json.dump(nuclear_trade_json, json_file, indent=4)